---
title: "HDA PySTAC-Client Introduction"
subtitle: "This notebook shows the basic use of DestinE Data Lake Harmonised Data Access using pystac-client."
author: "Author: Eumetsat"
tags: [STAC, Authentication, Access Token]
thumbnail: ../../img/STAC-01.png
license: MIT
copyright: "© 2024 EUMETSAT"
---
<div style="margin: 6px 0;">
  <a href="https://jupyter.central.data.destination-earth.eu/user-redirect/lab/tree/DestinE-DataLake-Lab/HDA/PySTAC/HDA-PyStac-Client.ipynb" target="_blank" style="text-decoration: none;">
    <span class="launch">🚀 Launch in JupyterHub</span>
  </a>
</div>

This notebook shows the basic use of DestinE Data Lake Harmonised Data Access using pystac-client.
It will include iterating through Collections and Items, and perform simple spatio-temporal searches.

## Obtain DEDL Access Token to use the HDA service

In [1]:
pip install --quiet --upgrade destinelab

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import json
import os
from getpass import getpass
import destinelab as deauth

In [3]:
DESP_USERNAME = input("Please input your DESP username or email: ")
DESP_PASSWORD = getpass("Please input your DESP password: ")

auth = deauth.AuthHandler(DESP_USERNAME, DESP_PASSWORD)
access_token = auth.get_token()
if access_token is not None:
    print("DEDL/DESP Access Token Obtained Successfully")
else:
    print("Failed to Obtain DEDL/DESP Access Token")

auth_headers = {"Authorization": f"Bearer {access_token}"}

Please input your DESP username or email:  eum-dedl-user
Please input your DESP password:  ········


Response code: 200
DEDL/DESP Access Token Obtained Successfully


## Set username and password as environment variables to be used for DEDL data access

In [4]:
import os

os.environ["EODAG__DEDL__AUTH__CREDENTIALS__USERNAME"] = DESP_USERNAME
os.environ["EODAG__DEDL__AUTH__CREDENTIALS__PASSWORD"] = DESP_PASSWORD

# Create pystac client object for HDA STAC API
We first connect to an API by retrieving the root catalog, or landing page, of the API with the Client.open function.

In [5]:
from pystac_client import Client

HDA_API_URL = "https://hda.data.destination-earth.eu/stac"
cat = Client.open(HDA_API_URL, headers=auth_headers)

## Query all available collections
As with a static catalog the get_collections function will iterate through the Collections in the Catalog. 
Notice that because this is an API it can get all the Collections through a single call, rather than having to fetch each one individually.

In [6]:
from rich.console import Console
import rich.table

console = Console()

hda_collections = cat.get_collections()

table = rich.table.Table(title="HDA collections", expand=True)
table.add_column("ID", style="cyan", justify="right",no_wrap=True)
table.add_column("Title", style="violet", no_wrap=True)
for collection in hda_collections:
    table.add_row(collection.id, collection.title)
console.print(table)

                                                  HDA collections                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                      ID ┃ Title                                                                 ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│                      EO.CLMS.DAT.CORINE │ CORINE Land Cover                                                     │
│               EO.CLMS.DAT.GLO.DMP300_V1 │ Dry Matter Productivity 2014-present (raster 300 m), global, 10-dail… │
│             EO.CLMS.DAT.GLO.FAPAR300_V1 │ Fraction of Absorbed Photosynthetically Active Radiation 2014-presen… │
│            EO.CLMS.DAT.GLO.FCOVER300_V1 │ Fraction of Green Vegetation Cover 2014-present (raster 300 m), glob… │
│              EO.CLMS.DAT.GLO.GDMP300_V1 │ Gross Dry Matter Productivity 2014-present (raster 300 m), global, 1… │
│               EO.CLMS.DAT.GLO.LAI300_V1 │ Leaf Area Index 2014-present (raster 300 m), global, 10-daily – vers… │
│              EO.CLMS.DAT.GLO.NDVI300_V1 │ Normalised Difference Vegetation Index 2014-2020 (raster 300 m), glo… │
│             EO.CLMS.DAT.GLO.NDVI_1KM_V2 │ Normalised Difference Vegetation Index 1998-2020 (raster 1 km), glob… │
│         EO.CLMS.DAT.SENTINEL-2.HRVPP.VI │ High Resolution Vegetation Phenology and Productivity: Vegetation In… │
│          EO.DEM.DAT.COP-DEM_GLO-30-DGED │ COPERNICUS Digital Elevation Model (DEM) 30 meters DGED               │
│          EO.DEM.DAT.COP-DEM_GLO-30-DTED │ COPERNICUS Digital Elevation Model (DEM) 30 meters DTED               │
│          EO.DEM.DAT.COP-DEM_GLO-90-DGED │ COPERNICUS Digital Elevation Model (DEM) 90 meters DGED               │
│          EO.DEM.DAT.COP-DEM_GLO-90-DTED │ COPERNICUS Digital Elevation Model (DEM) 90 meters DTED               │
│ EO.ECMWF.DAT.CAMS_EUROPE_AIR_QUALITY_F… │ CAMS European air quality forecasts                                   │
│ EO.ECMWF.DAT.CAMS_EUROPE_AIR_QUALITY_R… │ CAMS European air quality reanalyses                                  │
│ EO.ECMWF.DAT.CAMS_GLOBAL_ATMOSHERIC_CO… │ CAMS global atmospheric composition forecasts                         │
│ EO.ECMWF.DAT.CAMS_GLOBAL_EMISSION_INVE… │ CAMS global emission inventories                                      │
│ EO.ECMWF.DAT.CAMS_GLOBAL_FIRE_EMISSION… │ CAMS global biomass burning emissions based on fire radiative power … │
│ EO.ECMWF.DAT.CAMS_GLOBAL_GREENHOUSE_GA… │ CAMS global greenhouse gas reanalysis (EGG4)                          │
│ EO.ECMWF.DAT.CAMS_GLOBAL_GREENHOUSE_GA… │ CAMS global greenhouse gas reanalysis (EGG4) monthly averaged fields  │
│ EO.ECMWF.DAT.CAMS_GLOBAL_RADIATIVE_FOR… │ CAMS global radiative forcings                                        │
│ EO.ECMWF.DAT.CAMS_GLOBAL_RADIATIVE_FOR… │ CAMS global radiative forcing - auxilliary variables                  │
│ EO.ECMWF.DAT.CAMS_GLOBAL_REANALYSIS_EA… │ CAMS global reanalysis (EAC4)                                         │
│ EO.ECMWF.DAT.CAMS_GLOBAL_REANALYSIS_EA… │ CAMS global reanalysis (EAC4) monthly averaged fields                 │
│ EO.ECMWF.DAT.CAMS_GREENHOUSE_GAS_FLUXES │ CAMS global inversion-optimised greenhouse gas fluxes and concentrat… │
│ EO.ECMWF.DAT.CAMS_SOLAR_RADIATION_TIME… │ CAMS solar radiation time-series                                      │
│       EO.ECMWF.DAT.CEMS_FIRE_HISTORICAL │ Fire danger indices historical data from the Copernicus Emergency Ma… │
│       EO.ECMWF.DAT.CEMS_GLOFAS_FORECAST │ River discharge and related forecasted data by the European Flood Aw… │
│     EO.ECMWF.DAT.CEMS_GLOFAS_HISTORICAL │ River discharge and related historical data from the Global Flood Aw… │
│     EO.ECMWF.DAT.CEMS_GLOFAS_REFORECAST │ Reforecasts of river discharge and related data by the Global Flood … │
│       EO.ECMWF.DAT.CEMS_GLOFAS_SEASONAL │ Seasonal for

## Obtain provider information for each individual collection

In [7]:
table = rich.table.Table(title="HDA collections | Providers", expand=True)
table.add_column("Title", style="cyan", justify="right", no_wrap=True)
table.add_column("Provider", style="violet", no_wrap=True)

hda_collections = cat.get_collections()

for collection in hda_collections:
    collection_details = cat.get_collection(collection.id)
    provider = ','.join(str(x.name) for x in collection_details.providers)
    table.add_row(collection_details.title, provider)
console.print(table)

                                            HDA collections | Providers                                            
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                  Title ┃ Provider                                               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│                                      CORINE Land Cover │ European Environment Agency,wekeo                      │
│ Dry Matter Productivity 2014-present (raster 300 m), … │ VITO NV,European Commission Directorate-General for I… │
│ Fraction of Absorbed Photosynthetically Active Radiat… │ VITO NV,European Commission Directorate-General for I… │
│ Fraction of Green Vegetation Cover 2014-present (rast… │ VITO NV,European Commission Directorate-General for I… │
│ Gross Dry Matter Productivity 2014-present (raster 30… │ VITO NV,European Commission Directorate-General for I… │
│ Leaf Area Index 2014-present (raster 300 m), global, … │ VITO NV,European Commission Directorate-General for I… │
│ Normalised Difference Vegetation Index 2014-2020 (ras… │ VITO NV,European Commission Directorate-General for I… │
│ Normalised Difference Vegetation Index 1998-2020 (ras… │ VITO NV,European Commission Directorate-General for I… │
│ High Resolution Vegetation Phenology and Productivity… │ VITO NV,European Union represented by the European Co… │
│ COPERNICUS Digital Elevation Model (DEM) 30 meters DG… │ ESA,dedl,creodias,wekeo                                │
│ COPERNICUS Digital Elevation Model (DEM) 30 meters DT… │ ESA,dedl,creodias,wekeo                                │
│ COPERNICUS Digital Elevation Model (DEM) 90 meters DG… │ ESA,dedl,creodias,wekeo                                │
│ COPERNICUS Digital Elevation Model (DEM) 90 meters DT… │ ESA,dedl,creodias,wekeo                                │
│                    CAMS European air quality forecasts │ European Centre for Medium-Range Weather Forecasts (E… │
│                   CAMS European air quality reanalyses │ European Centre for Medium-Range Weather Forecasts (E… │
│          CAMS global atmospheric composition forecasts │ European Centre for Medium-Range Weather Forecasts (E… │
│                       CAMS global emission inventories │ European Centre for Medium-Range Weather Forecasts (E… │
│ CAMS global biomass burning emissions based on fire r… │ European Centre for Medium-Range Weather Forecasts (E… │
│           CAMS global greenhouse gas reanalysis (EGG4) │ European Centre for Medium-Range Weather Forecasts (E… │
│ CAMS global greenhouse gas reanalysis (EGG4) monthly … │ European Centre for Medium-Range Weather Forecasts (E… │
│                         CAMS global radiative forcings │ European Centre for Medium-Range Weather Forecasts (E… │
│   CAMS global radiative forcing - auxilliary variables │ European Centre for Medium-Range Weather Forecasts (E… │
│                          CAMS global reanalysis (EAC4) │ European Centre for Medium-Range Weather Forecasts (E… │
│  CAMS global reanalysis (EAC4) monthly averaged fields │ European Centre for Medium-Range Weather Forecasts (E… │
│ CAMS global inversion-optimised greenhouse gas fluxes… │ European Centre for Medium-Range Weather Forecasts (E… │
│                       CAMS solar radiation time-series │ European Centre for Medium-Range Weather Forecasts (E… │
│ Fire danger indices historical data from the Copernic… │ European Centre for Medium-Range Weather Forecasts (E… │
│ River discharge and related forecasted data by the Eu… │ European Centre for Medium-Range Weather Forecasts (E… │
│ River discharge and related historical data from the … │ European Centre for Medium-Range Weather Forecasts (E… │
│ Reforecasts of river discharge and related data by th… │ European Centre for Medium-Range Weather Forecasts (E… │
│ Seasonal forecasts of river discharge and related dat…

## Inspect Items of a Collection
The main functions for getting items return iterators, where pystac-client will handle retrieval of additional pages when needed. Note that one request is made for the first ten items, then a second request for the next ten.

In [9]:
coll_name = 'EO.ESA.DAT.SENTINEL-1.L1_GRD'
search = cat.search(
    max_items=10,
    collections=[coll_name],
    bbox=[-72.5,40.5,-72,41],
    datetime="2023-09-09T00:00:00Z/2023-09-20T23:59:59Z"
)

coll_items = search.item_collection()
console.print(f"For collection {coll_name} we found {len(coll_items)} items")

For collection EO.ESA.DAT.SENTINEL-1.L1_GRD we found 4 items

In [10]:
import geopandas

df = geopandas.GeoDataFrame.from_features(coll_items.to_dict(), crs="epsg:4326")
df.head()



,geometry,providers,datetime,start_datetime,end_datetime,updated,description,license,constellation,platform,...,dedl:productComposition,dedl:processorVersion,dedl:totalSlices,dedl:timeliness,dedl:productIdentifier,dedl:segmentStartTime,dedl:beginningDateTime,dedl:datatakeID,dedl:sliceProductFlag,dedl:uid
0,"POLYGON ((-72.73724 38.96255, -69.78494 39.365...","[{'name': 'dedl', 'description': 'DestineE Dat...",2023-09-09T22:43:00.442715Z,2023-09-09T22:43:00.442715Z,2023-09-09T22:43:25.441630Z,2023-09-10T03:08:53.951840Z,The [Sentinel-1](https://sentinel.esa.int/web/...,proprietary,SENTINEL-1,A,...,Slice,3.61,16,Fast-24h,/eodata/Sentinel-1/SAR/IW_GRDH_1S/2023/09/09/S...,2023-09-09T22:41:16.742000+00:00,2023-09-09T22:43:00.442715Z,396478,False,64caac74-8c5b-4f4d-9d3d-2d029b6e1ddc
1,"POLYGON ((-73.11689 40.46453, -70.09798 40.866...","[{'name': 'dedl', 'description': 'DestineE Dat...",2023-09-09T22:43:25.443129Z,2023-09-09T22:43:25.443129Z,2023-09-09T22:43:50.442045Z,2023-09-10T02:57:33.550132Z,The [Sentinel-1](https://sentinel.esa.int/web/...,proprietary,SENTINEL-1,A,...,Slice,3.61,16,Fast-24h,/eodata/Sentinel-1/SAR/IW_GRDH_1S/2023/09/09/S...,2023-09-09T22:41:16.742000+00:00,2023-09-09T22:43:25.443129Z,396478,False,f97e0469-9ecc-466e-9706-f3a0a98a8426
2,"POLYGON ((-74.73586 38.73753, -71.84171 39.134...","[{'name': 'dedl', 'description': 'DestineE Dat...",2023-09-14T22:51:10.454096Z,2023-09-14T22:51:10.454096Z,2023-09-14T22:51:35.452004Z,2023-09-15T01:30:12.125014Z,The [Sentinel-1](https://sentinel.esa.int/web/...,proprietary,SENTINEL-1,A,...,Slice,3.61,7,Fast-24h,/eodata/Sentinel-1/SAR/IW_GRDH_1S/2023/09/14/S...,2023-09-14T22:49:26.753000+00:00,2023-09-14T22:51:10.454096Z,397120,False,e81554b4-5784-4f0b-ad69-ef984b489b52
3,"POLYGON ((-75.11328 40.2397, -72.15421 40.6353...","[{'name': 'dedl', 'description': 'DestineE Dat...",2023-09-14T22:51:35.453501Z,2023-09-14T22:51:35.453501Z,2023-09-14T22:52:00.452905Z,2023-09-15T01:28:26.349235Z,The [Sentinel-1](https://sentinel.esa.int/web/...,proprietary,SENTINEL-1,A,...,Slice,3.61,7,Fast-24h,/eodata/Sentinel-1/SAR/IW_GRDH_1S/2023/09/14/S...,2023-09-14T22:49:26.753000+00:00,2023-09-14T22:51:35.453501Z,397120,False,ac9a45a9-e210-4d84-824c-f1fe12e7a4ca


## Inspect STAC assets of an item

In [11]:
import rich.table

selected_item = coll_items[3]

table = rich.table.Table(title="Assets in STAC Item")
table.add_column("Asset Key", style="cyan", no_wrap=True)
table.add_column("Description")
for asset_key, asset in selected_item.assets.items():
    table.add_row(asset_key, asset.title)

console.print(table)

                                                Assets in STAC Item                                                
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ Asset Key                                                                                           ┃ Descript… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ downloadLink                                                                                        │ Download  │
│                                                                                                     │ link      │
│ S1A_IW_GRDH_1SDV_20230914T225135_20230914T225200_050330_060F40_9F54.SAFE-report-20230915T011838.pdf │ S1A_IW_G… │
│ calibration-s1a-iw-grd-vh-20230914t225135-20230914t225200-050330-060f40-002.xml                     │ calibrat… │
│ calibration-s1a-iw-grd-vv-20230914t225135-20230914t225200-050330-060f40-001.xml                     │ calibrat… │
│ noise-s1a-iw-grd-vh-20230914t225135-20230914t225200-050330-060f40-002.xml                           │ noise-s1… │
│ noise-s1a-iw-grd-vv-20230914t225135-20230914t225200-050330-060f40-001.xml                           │ noise-s1… │
│ rfi-s1a-iw-grd-vh-20230914t225135-20230914t225200-050330-060f40-002.xml                             │ rfi-s1a-… │
│ rfi-s1a-iw-grd-vv-20230914t225135-20230914t225200-050330-060f40-001.xml                             │ rfi-s1a-… │
│ s1a-iw-grd-vh-20230914t225135-20230914t225200-050330-060f40-002.xml                                 │ s1a-iw-g… │
│ s1a-iw-grd-vv-20230914t225135-20230914t225200-050330-060f40-001.xml                                 │ s1a-iw-g… │
│ manifest.safe                                                                                       │ manifest… │
│ s1a-iw-grd-vh-20230914t225135-20230914t225200-050330-060f40-002.tiff                                │ s1a-iw-g… │
│ s1a-iw-grd-vv-20230914t225135-20230914t225200-050330-060f40-001.tiff                                │ s1a-iw-g… │
│ logo.png                                                                                            │ logo.png  │
│ map-overlay.kml                                                                                     │ map-over… │
│ product-preview.html                                                                                │ product-… │
│ quick-look.png                                                                                      │ quick-lo… │
│ thumbnail.png                                                                                       │ thumbnai… │
│ s1-level-1-calibration.xsd                                                                          │ s1-level… │
│ s1-level-1-measurement.xsd                                                                          │ s1-level… │
│ s1-level-1-noise.xsd                                                                                │ s1-level… │
│ s1-level-1-product.xsd                                                                              │ s1-level… │
│ s1-level-1-quicklook.xsd                                                                            │ s1-level… │
│ s1-level-1-rfi.xsd                                                                                  │ s1-level… │
│ s1-map-overlay.xsd                                                                                  │ s1-map-o… │
│ s1-object-types.xsd                                                                                 │ s1-objec… │
│ s1-product-preview.xsd                                                                              │ s1-produ… │
│ thumbnail                                                                                           │ Thumbnail │
└─────────────────────────────────────────────────────────────────────────────────────────────────────┴───────────┘

In [12]:
from IPython.display import Image

Image(url=selected_item.assets["thumbnail"].href, width=500)

In [13]:
down_uri = selected_item.assets["downloadLink"].href
console.print(f"Download link of asset is {down_uri}")

Download link of asset is 
https://hda.data.destination-earth.eu/stac/collections/EO.ESA.DAT.SENTINEL-1.L1_GRD/items/S1A_IW_GRDH_1SDV_20230914
T225135_20230914T225200_050330_060F40_9F54/download?provider=dedl

### Download asset to JupyterLab

In [14]:
selected_item.id

'S1A_IW_GRDH_1SDV_20230914T225135_20230914T225200_050330_060F40_9F54'

In [15]:
selected_item.assets["downloadLink"]

<Asset href=https://hda.data.destination-earth.eu/stac/collections/EO.ESA.DAT.SENTINEL-1.L1_GRD/items/S1A_IW_GRDH_1SDV_20230914T225135_20230914T225200_050330_060F40_9F54/download?provider=dedl>

In [16]:
# Make http request for remote file data
data = requests.get(selected_item.assets["downloadLink"].href,
                   headers=auth_headers)
mtype = selected_item.assets["downloadLink"].media_type.split("/")[1]
# Save file data to local copy
with open(f"{selected_item.id}.{mtype}", 'wb')as file:
    file.write(data.content)